In [1]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.support.select import Select
import selenium.webdriver.support.ui as UI
import numpy as np
from ipywidgets.widgets import BoundedIntText, Label, Button, Text, HTML
from IPython.display import display, clear_output
from IPython.display import display, FileLink
from ipywidgets.widgets import Layout, HBox
import ipywidgets as widgets
from ipywidgets import HTML
import matplotlib as plt
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [56]:
SIRET = Text(
    value='',
    placeholder='Type something',
    description='SIRET:',
    disabled=False
)

instruction=HTML('Préciser numéro de SIRET et appuyer sur enter')
output = widgets.Output()
clear_output()

display(instruction)
display(SIRET, output)

def handle_submit(sender):
    with output:
        url='https://www.societe.com/'
        options = webdriver.ChromeOptions()
        options.binary_location = os.environ.get("GOOGLE_CHROME_BIN")
        options.add_argument("--headless")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--no-sandbox")
        driver = webdriver.Chrome(executable_path=os.environ.get("CHROMEDRIVER_PATH"), options=options)
        driver.get(url)
        time.sleep(1)
        
        driver.maximize_window()
        time.sleep(1)
    
        accept_button = driver.find_element_by_xpath('//*[@id="didomi-notice-agree-button"]')
        accept_button.click()
    
        search_bar = driver.find_element_by_xpath('//*[@id="input_search"]')
        search_bar.send_keys(SIRET.value)
        look_button = driver.find_element_by_xpath('//*[@id="buttsearch"]')
        look_button.click()
        
        time.sleep(1)

        element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="menuentreprise_desktop"]/ul/li[8]/div/a')))
        element.click();
        time.sleep(1)
    
        select = UI.Select(driver.find_element_by_xpath('//*[@id="selectsociaux"]'))
    
        list_option=[]
        for option in select.options:
            list_option.append(option.text)  
    
        list_numbers=list(range(len(list_option)))
        final_list_numbers=list_numbers[0::2]
        final_list_numbers.pop(0)
        list_years=[]
        for i in final_list_numbers:
            list_years.append(list_option[i])
    
        name=driver.find_element_by_xpath('//*[@id="identite_deno"]').text
        year_1 = list_option[0]
    
        assets_object = driver.find_element_by_xpath('//*[@id="actif"]').get_attribute("outerHTML")
        assets_intermediary = pd.read_html(assets_object)
        assets = assets_intermediary[0]
        assets_final=assets.drop(columns='Variation', index=[0,1,6,11]).reset_index(drop=True)
    
        liabilities_object = driver.find_element_by_xpath('//*[@id="passif"]').get_attribute("outerHTML")
        liabilities_intermediary = pd.read_html(liabilities_object)
        liabilities = liabilities_intermediary[0]
        liabilities_final=liabilities.drop(columns='Variation', index=[0,1,3]).reset_index(drop=True)
    
        pandl_object = driver.find_element_by_xpath('//*[@id="compteresultat-ctr"]').get_attribute("outerHTML")
        pandl_intermediary = pd.read_html(pandl_object)
        pandl = pandl_intermediary[0]
        pandl_second_intermdiary=pandl.drop(columns='Variation', index=[0,1,6,10,12]).reset_index(drop=True)
        pandl_final=pandl_second_intermdiary.rename(columns={'Date': 'Date de clôture'})
    
        frames_of_financial_data = [assets_final,liabilities_final,pandl_final]
        financial_data = pd.concat(frames_of_financial_data).reset_index(drop=True)
    
        #DOWNLOADING ALL FINANCIAL DATA
    
        time.sleep(1)
        name=driver.find_element_by_xpath('//*[@id="identite_deno"]').text
    
        li = [financial_data]
    
        for i in list_years:
            select2 = Select(driver.find_element_by_xpath('//*[@id="selectsociaux"]')).select_by_visible_text(i)
            time.sleep(1)
            assets_object_loop = driver.find_element_by_xpath('//*[@id="actif"]').get_attribute("outerHTML")
            assets_intermediary_loop = pd.read_html(assets_object_loop)
            assets_loop = assets_intermediary_loop[0]
            assets_final_loop=assets_loop.drop(columns='Variation', index=[0,1,6,11], errors='ignore').reset_index(drop=True)
        
            liabilities_object_loop = driver.find_element_by_xpath('//*[@id="passif"]').get_attribute("outerHTML")
            liabilities_intermediary_loop = pd.read_html(liabilities_object_loop)
            liabilities_loop = liabilities_intermediary_loop[0]
            liabilities_final_loop=liabilities_loop.drop(columns='Variation', index=[0,1,3], errors='ignore').reset_index(drop=True)
        
            pandl_object_loop = driver.find_element_by_xpath('//*[@id="compteresultat-ctr"]').get_attribute("outerHTML")
            pandl_intermediary_loop = pd.read_html(pandl_object_loop)
            pandl_loop = pandl_intermediary_loop[0]
            pandl_second_intermdiary_loop=pandl_loop.drop(columns='Variation', index=[0,1,6,10,12], errors='ignore').reset_index(drop=True)
            pandl_final_loop=pandl_second_intermdiary_loop.rename(columns={'Date': 'Date de clôture'})
            frames_of_financial_data_loop = [assets_final_loop,liabilities_final_loop,pandl_final_loop]
            financial_data_loop_intermediate = pd.concat(frames_of_financial_data_loop).reset_index(drop=True)
            financial_data_loop = financial_data_loop_intermediate.drop(columns='Date de clôture')
        
            li.append(financial_data_loop)
    
        first_frame = pd.concat(li, axis=1)
    
        def get_rid(x):
            try:
                return x.replace(u'\xa0',u'')
            except AttributeError:
                return np.nan
        second_frame=first_frame.applymap(lambda x: get_rid(x))
    
        columns = second_frame.columns.tolist()
        columns = columns[::-1]
        columns = [columns[-1]] + columns[:-1]
        third_frame = second_frame[columns]
    
        def float_function(x):
            try:
                return float(x)
            except ValueError:
                return x
    
        final_frame=third_frame.applymap(lambda x: float_function(x))
        driver.quit()
    
        def millions(x, pos):
            'The two args are the value and tick position'
            return '%1.1fM' % (x * 1e-6)


        formatter = FuncFormatter(millions)

        dates=list(final_frame.columns)
        del dates[0]
        chiffre_d_affaires = final_frame.iloc[18].to_list()
        del chiffre_d_affaires[0]
        EBITDA = final_frame.iloc[22].to_list()
        del EBITDA[0]

        fig = plt.figure()
        ax = fig.add_subplot(111)
        plt.xticks(rotation=90)
        lns1=ax.bar(dates,chiffre_d_affaires, color='#3633D6', label="Chiffre d'Affaires")
        ax.yaxis.set_major_formatter(formatter)

        ax2=ax.twinx()
        lns2=ax2.plot(dates, EBITDA, color='#BF533B', label='EBITDA', linewidth=4)
        ax2.yaxis.set_major_formatter(formatter)

        plt.legend(handles=[lns1,lns2[0]])

        ax.set_title("CA - "+name)
        ax.set_ylabel('CA')
        ax2.set_ylabel('EBITDA')
        display(plt.show())
    
SIRET.on_submit(handle_submit)

HTML(value='Préciser numéro de SIRET et appuyer sur enter')

Text(value='', description='SIRET:', placeholder='Type something')

Output()